[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/llamaindex/cookbook/blob/main/integrations/Klavis/Use_Klavis_with_LlamaIndex.ipynb)


# LlamaIndex + Klavis AI Integration

This tutorial demonstrates how to build AI agents using LlamaIndex's agent framework with Klavis MCP (Model Context Protocol) servers for enhanced functionality. 


## Prerequisites

Before we begin, you'll need:

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [1]:
# Install the required packages
%pip install -qU llama-index llama-index-tools-mcp requests


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Set environment variables
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXX"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "XXXXXXXXXXXX"  # Replace with your actual Klavis API key


## Set up Klavis API

Klavis API is built on REST principles. Check out the [documentation](https://docs.klavis.ai/api-reference/introduction) if you're interested in learning more.

In [4]:
import requests
from typing import Dict, Any
import webbrowser

class KlavisAPI:
    """API Client for Klavis API."""
    
    def __init__(self, api_key: str, base_url: str = "https://api.klavis.ai"):
        self.api_key = api_key
        self.base_url = base_url
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
    
    def _make_request(self, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
        """Make HTTP request with error handling."""
        url = f"{self.base_url}{endpoint}"
        response = requests.request(method, url, headers=self.headers, **kwargs)
        response.raise_for_status()
        return response.json()
    
    def create_mcp_instance(self, server_name: str, user_id: str, platform_name: str) -> Dict[str, str]:
        """Create MCP server instance."""
        data = {
            "serverName": server_name,
            "userId": user_id,
            "platformName": platform_name,
            "connectionType": "StreamableHttp"
        }
        result = self._make_request("POST", "/mcp-server/instance/create", json=data)
        print(f"✅ Created {server_name} MCP instance")
        return {
            'serverUrl': result['serverUrl'],
            'instanceId': result['instanceId']
        }
    
    # Advanced use case if the MCP server requires OAuth authorization
    def redirect_to_oauth(self, instance_id: str, server_name: str) -> None:
        """Open OAuth authorization URL in browser."""
        oauth_url = f"{self.base_url}/oauth/{server_name.lower()}/authorize?instance_id={instance_id}"
        print(f"🔐 Opening OAuth authorization for {server_name}")
        print(f"If you are not redirected automatically, please open this URL: {oauth_url}")
        webbrowser.open(oauth_url)


## Case 1: YouTube AI Agent

Create an AI agent to summarize YouTube videos using LlamaIndex and Klavis MCP Server.

In [10]:
from llama_index.llms.openai import OpenAI
from llama_index.tools.mcp import BasicMCPClient

# 1. Initialize LLM and Klavis API client
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
klavis_api_client = KlavisAPI(api_key=os.getenv("KLAVIS_API_KEY"))

# 2. using Klavis API - Create a YouTube MCP server and get the server URL
youtube_mcp_instance = klavis_api_client.create_mcp_instance(
    server_name="YouTube",
    user_id="1234",
    platform_name="Klavis",
)

youtube_mcp_server_url = youtube_mcp_instance["serverUrl"]
print(f"🔗 YouTube MCP server created!")

# 3. using LlamaIndex - Create a MCP client
youtube_mcp_client = BasicMCPClient(youtube_mcp_server_url)


✅ Created YouTube MCP instance
🔗 YouTube MCP server created!


In [7]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.tools.mcp import (
    get_tools_from_mcp_url,
    aget_tools_from_mcp_url,
)
youtube_tools = await aget_tools_from_mcp_url(
    youtube_mcp_server_url,
    client=youtube_mcp_client,
)

# 4. using LlamaIndex - Create a YouTube agent
youtube_agent = FunctionAgent(
    name="youtube_agent",
    description="Agent using MCP-based tools",
    tools=youtube_tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools."
)


YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=MmiveeGxfX0&t=528s"  # Replace with your video URL
response = await youtube_agent.run(f"Summarize this video: {YOUTUBE_VIDEO_URL}")
print(response)

The video titled "Introducing AgentWorkflow, a way to easily create multi-agent systems in Llamaindex" discusses a new system called AgentWorkflow designed for building and orchestrating AI agent systems. Here are the key points covered in the video:

- **Overview of AgentWorkflow**: It helps coordinate multiple AI agents while maintaining state and context, making it suitable for both single specialized agents and teams working together.

- **Key Features**:
  - Flexible agent types, including FunctionAgent and ReActAgent.
  - Built-in state management for better control.
  - Real-time monitoring capabilities.
  - Human-in-the-loop functionalities for enhanced interaction.

- **Learning Resources**: The video encourages viewers to explore comprehensive tutorials and documentation to learn how to build everything from simple assistants to complex multi-agent systems.

- **Links Provided**:
  - Basic Tutorial
  - Full documentation
  - Introductory blog post
  - Discord community link



✅ Nice work! You’ve successfully summarized your favorite YouTube video and saved tons of time!

## Case 2: Multi-Agent Workflow

Build a LlamaIndex AgentWorkflow that summarizes YouTube videos and sends the summary via email.

In [11]:
from llama_index.llms.openai import OpenAI

# 1. Initialize LLM and Klavis API client
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
klavis_api_client = KlavisAPI(api_key=os.getenv("KLAVIS_API_KEY"))

# 2. Create a YouTube MCP server     
youtube_mcp_instance = klavis_api_client.create_mcp_instance(
    server_name="YouTube",
    user_id="1234",
    platform_name="Klavis",
)

# 3. Create a Gmail MCP server with OAuth authorization
gmail_mcp_instance = klavis_api_client.create_mcp_instance(
    server_name="Gmail",
    user_id="1234",
    platform_name="Klavis",
)
klavis_api_client.redirect_to_oauth(gmail_mcp_instance["instanceId"], "Gmail")

✅ Created YouTube MCP instance
✅ Created Gmail MCP instance
🔐 Opening OAuth authorization for Gmail
If you are not redirected automatically, please open this URL: https://api.klavis.ai/oauth/gmail/authorize?instance_id=d9d482b3-433a-4330-9a8b-9548c0b0a326


In [12]:
from llama_index.tools.mcp import (
    BasicMCPClient,
    get_tools_from_mcp_url,
    aget_tools_from_mcp_url,
)

# using LlamaIndex mcp framework - tool format transformation and setting up the MCP Client

youtube_mcp_server_url = youtube_mcp_instance["serverUrl"]
gmail_mcp_server_url = gmail_mcp_instance["serverUrl"]

youtube_mcp_client = BasicMCPClient(youtube_mcp_server_url)
gmail_mcp_client = BasicMCPClient(gmail_mcp_server_url)

youtube_tools = await aget_tools_from_mcp_url(
    youtube_mcp_server_url,
    client=youtube_mcp_client,
)

gmail_tools = await aget_tools_from_mcp_url(
    gmail_mcp_server_url,
    client=gmail_mcp_client,
)


In [13]:
from llama_index.core.agent.workflow import FunctionAgent

# using LlamaIndex agent framework - create agents

youtube_agent = FunctionAgent(
    name="youtube_agent",
    description="Agent using MCP-based tools",
    tools=youtube_tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools.",
    can_handoff_to=["gmail_agent"], # handoff to gmail agent
)

gmail_agent = FunctionAgent(
    name="gmail_agent",
    description="Agent using MCP-based tools",
    tools=gmail_tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools."
)

In [14]:
from llama_index.core.agent.workflow import AgentWorkflow


# using LlamaIndex workflow framework - create a workflow with multiple agents

workflow = AgentWorkflow(
        agents=[youtube_agent, gmail_agent],
        root_agent="youtube_agent",
    )

In [17]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=MmiveeGxfX0&t=528s" # pick a video you like!
EMAIL_RECIPIENT = "zihaolin@klavis.ai"  # Replace with your email

resp = await workflow.run(user_msg=f"Summarize this video {YOUTUBE_VIDEO_URL} and send it to {EMAIL_RECIPIENT}")
print("\n✅ Report:\n", resp.response.content)


✅ Report:
 The summary of the video has been successfully sent to zihaolin@klavis.ai. If you need anything else, feel free to ask!



## Summary

In this tutorial, we explored how to integrate LlamaIndex with Klavis AI to build powerful AI agents using MCP (Model Context Protocol) servers. Here's what we accomplished:

### Key Takeaways:

1. **Single Agent Setup**: Created a YouTube AI agent that can summarize videos using the Klavis YouTube MCP server
2. **Multi-Agent Workflow**: Built a sophisticated workflow combining YouTube and Gmail agents to summarize videos and automatically send summaries via email
3. **MCP Integration**: Learned how to use Klavis MCP servers with LlamaIndex's agent framework for enhanced functionality

This integration opens up endless possibilities for building AI agents that can interact with various services and platforms through Klavis MCP servers. You can now create agents that work with YouTube, Gmail, GitHub, Slack, and many other services supported by Klavis.

Happy building! 🚀